In [2]:
import numpy as np
import pandas as pd
import re

file = open('data/wiki sample data.txt','r')

lines = file.readlines()
#print (lines)

df = pd.DataFrame(columns=['timestamp','user','minorEdit','pageLength','editDiff','comment','tags'])
i=0
for line in lines:
        #split via brackets
        token = re.split('\(|\)',line)
        #print(token)
        timestamp = token[2].split('?')[0]
        timestamp = pd.to_datetime(timestamp)
        #split by ?
        user = token[2].split('?')[1:]
        user = ''.join(user)
        user = user[1:-1]
        m = token[4]
        m= m.split('.')[0]
        m= True if (m=='? m ') else False
        byte = token[5].split(' ')[0]
        byte = int(byte.replace(',',''))
        change = token[7]
        change = int(change.replace(',',''))
        comment = re.split('\. \. ',line)[-1]
        tagComment = comment.split('(undo)')
        #print (tagComment)
        comment = tagComment[0][1:-2]
        #undo is not at the end
        if (len(tagComment)>1):
            tag = tagComment[1][2:-2]
            tag = tag.split(':')
            if (len(tag)>1):
                tag = tag[1][1:]
                tag = tag.split(',')
            if (tag[0]==''):
                tag =[]
        else:
            tag=''
        
        #tag = tag.split(',')
        #print (tag)
        
        df.loc[i]=[timestamp,user,m,byte,change,comment,tag]
        i+=1

print (df.shape)

(1338, 7)


In [29]:


POI1 = df['user'].mode().astype(str)
#print (POI1)

mask = df['user']=='DinoBambinoNFS'
POI1df = df[mask]['comment']
#print (POI1df)

commentsPOI1 = list(POI1df)
#print (commentsPOI1)

In [35]:
import nltk
import re
commentsDf = df['comment']

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

def tokenizeAndStem(text):
    CommentsTokens=[]
    #for userComments in commentsDf:
    CommentsTokens = (nltk.word_tokenize(text))
    #filter out punctuations and numeric tokens
    filtered_tokens = []
    for token in CommentsTokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

totalvocab_stemmed = []
for i in commentsDf:
    stem = tokenizeAndStem(i)
    totalvocab_stemmed.extend(stem)

print (len(totalvocab_stemmed))

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=0.8,
                                 min_df=0, stop_words='english',
                                 use_idf=True,tokenizer=tokenizeAndStem, ngram_range=(1,3))

tfidf_matrix = tfidf_vectorizer.fit_transform(commentsDf)
print (tfidf_matrix.shape)
terms = tfidf_vectorizer.get_feature_names()
#print (terms)

from sklearn.cluster import KMeans

num_clusters = 5

km = KMeans(n_clusters=num_clusters)
km.fit(tfidf_matrix)

clusters = km.labels_.tolist()
#print (clusters)



7056
(1338, 5959)


In [50]:
def get_colors(clusters):
    colors=[]
    for i in clusters:
        if i==0:
            colors.append('red')
        if i==1:
            colors.append('blue')
        if i==2:
            colors.append('green')
        if i==3:
            colors.append('brown')
        if i==4:
            colors.append('yellow')
    return colors

colors = get_colors(clusters)

from bokeh.plotting import figure
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource, Select

output_notebook()
a=figure(x_range =list(set(list(df['user']))),plot_height=500, plot_width=900, title='clustering groups based on edits')
data = ColumnDataSource(data=dict(x=df['user'],y=df['timestamp'],colors = colors))
a.square('x','y',fill_color='colors',line_color='colors',source=data,size=10)
show(a)

Loading BokehJS ...